In [18]:
import pickle
import pandas as pd
import numpy as np
import requests
import calc

In [19]:
filename = 'btcusdr_lightgbm_model.pkl'
# 保存したモデルの読み込み
model = pickle.load(open(filename, 'rb'))

In [20]:
rsp = requests.get('https://api.bybit.com/v5/market/kline?category=linear&symbol=BTCUSDT&interval=1').json()
results = rsp['result']
df = pd.DataFrame(results['list'],columns=['startTime','Open','High','Low','Close','Volume','turnover'])

In [21]:
df["startTime"]=df["startTime"].astype(float)
df["Open"]=df["Open"].astype(float)
df["High"]=df["High"].astype(float)
df["Low"]=df["Low"].astype(float)
df["Close"]=df["Close"].astype(float)
df["Volume"]=df["Volume"].astype(float)

In [22]:
# 先頭行を削除
df = df.drop(df.index[0])
df.reset_index(inplace=True, drop=True)

# 時間でソート
df = df.sort_values('startTime')

# Timestamp列を追加
df['Timestamp'] = pd.to_datetime(df['startTime'], unit='ms')


In [23]:
# 短期および長期のSMAを計算して新たなカラムとして追加
short_term_window = 5  # 短期SMAのウィンドウ期間（例：5分）
long_term_window = 20  # 長期SMAのウィンドウ期間（例：20分）

df['SMA_short_term'] = calc.calculate_sma(df, 'Close', short_term_window)
df['SMA_long_term'] = calc.calculate_sma(df, 'Close', long_term_window)

# 結果を表示
print(df.tail())

      startTime     Open     High      Low    Close   Volume      turnover  \
4  1.699197e+12  34898.0  34899.5  34888.0  34891.0   16.133   562960.3632   
3  1.699197e+12  34891.0  34898.1  34874.9  34879.8  132.443  4620298.8349   
2  1.699197e+12  34879.8  34889.8  34879.8  34889.7    8.055    281000.552   
1  1.699197e+12  34889.7  34891.9  34886.5  34891.9   18.847   657552.5822   
0  1.699197e+12  34891.9  34891.9  34879.8  34882.3   46.626  1626579.7096   

            Timestamp  SMA_short_term  SMA_long_term  
4 2023-11-05 15:11:00        34896.40      34914.275  
3 2023-11-05 15:12:00        34887.36      34910.875  
2 2023-11-05 15:13:00        34887.30      34908.195  
1 2023-11-05 15:14:00        34890.08      34905.660  
0 2023-11-05 15:15:00        34886.94      34903.495  


In [24]:
# ボリンジャーバンドの上限、下限、中心線を計算して新たなカラムとして追加
window = 20  # ボリンジャーバンドのウィンドウ期間（例：20分）
num_std_dev = 2  # 標準偏差の倍数（通常は2）

df['BB_upper'], df['BB_lower'], df['BB_middle'] = calc.calculate_bollinger_bands(df, 'Close', window, num_std_dev)

# 結果を表示
print(df.tail())

      startTime     Open     High      Low    Close   Volume      turnover  \
4  1.699197e+12  34898.0  34899.5  34888.0  34891.0   16.133   562960.3632   
3  1.699197e+12  34891.0  34898.1  34874.9  34879.8  132.443  4620298.8349   
2  1.699197e+12  34879.8  34889.8  34879.8  34889.7    8.055    281000.552   
1  1.699197e+12  34889.7  34891.9  34886.5  34891.9   18.847   657552.5822   
0  1.699197e+12  34891.9  34891.9  34879.8  34882.3   46.626  1626579.7096   

            Timestamp  SMA_short_term  SMA_long_term      BB_upper  \
4 2023-11-05 15:11:00        34896.40      34914.275  34956.362422   
3 2023-11-05 15:12:00        34887.36      34910.875  34952.543662   
2 2023-11-05 15:13:00        34887.30      34908.195  34947.932744   
1 2023-11-05 15:14:00        34890.08      34905.660  34942.520956   
0 2023-11-05 15:15:00        34886.94      34903.495  34940.510813   

       BB_lower  BB_middle  
4  34872.187578  34914.275  
3  34869.206338  34910.875  
2  34868.457256  34908.

In [25]:
df.dtypes

startTime                float64
Open                     float64
High                     float64
Low                      float64
Close                    float64
Volume                   float64
turnover                  object
Timestamp         datetime64[ns]
SMA_short_term           float64
SMA_long_term            float64
BB_upper                 float64
BB_lower                 float64
BB_middle                float64
dtype: object

In [26]:
# RSIを計算して新たなカラムとして追加
window = 14  # RSIのウィンドウ期間（例：14分）

df['RSI'] = calc.calculate_rsi(df, 'Close', window)

# 結果を表示
print(df.tail())

      startTime     Open     High      Low    Close   Volume      turnover  \
4  1.699197e+12  34898.0  34899.5  34888.0  34891.0   16.133   562960.3632   
3  1.699197e+12  34891.0  34898.1  34874.9  34879.8  132.443  4620298.8349   
2  1.699197e+12  34879.8  34889.8  34879.8  34889.7    8.055    281000.552   
1  1.699197e+12  34889.7  34891.9  34886.5  34891.9   18.847   657552.5822   
0  1.699197e+12  34891.9  34891.9  34879.8  34882.3   46.626  1626579.7096   

            Timestamp  SMA_short_term  SMA_long_term      BB_upper  \
4 2023-11-05 15:11:00        34896.40      34914.275  34956.362422   
3 2023-11-05 15:12:00        34887.36      34910.875  34952.543662   
2 2023-11-05 15:13:00        34887.30      34908.195  34947.932744   
1 2023-11-05 15:14:00        34890.08      34905.660  34942.520956   
0 2023-11-05 15:15:00        34886.94      34903.495  34940.510813   

       BB_lower  BB_middle        RSI  
4  34872.187578  34914.275  37.359551  
3  34869.206338  34910.875  38

In [27]:
# MACDおよびシグナルラインを計算して新たなカラムとして追加
short_window = 12  # 短期EMAのウィンドウ期間（例：12分）
long_window = 26   # 長期EMAのウィンドウ期間（例：26分）
signal_window = 9  # シグナルラインのウィンドウ期間（例：9分）

df['MACD'], df['Signal_line'] = calc.calculate_macd(df, 'Close', short_window, long_window, signal_window)

# 結果を表示
print(df.tail())

      startTime     Open     High      Low    Close   Volume      turnover  \
4  1.699197e+12  34898.0  34899.5  34888.0  34891.0   16.133   562960.3632   
3  1.699197e+12  34891.0  34898.1  34874.9  34879.8  132.443  4620298.8349   
2  1.699197e+12  34879.8  34889.8  34879.8  34889.7    8.055    281000.552   
1  1.699197e+12  34889.7  34891.9  34886.5  34891.9   18.847   657552.5822   
0  1.699197e+12  34891.9  34891.9  34879.8  34882.3   46.626  1626579.7096   

            Timestamp  SMA_short_term  SMA_long_term      BB_upper  \
4 2023-11-05 15:11:00        34896.40      34914.275  34956.362422   
3 2023-11-05 15:12:00        34887.36      34910.875  34952.543662   
2 2023-11-05 15:13:00        34887.30      34908.195  34947.932744   
1 2023-11-05 15:14:00        34890.08      34905.660  34942.520956   
0 2023-11-05 15:15:00        34886.94      34903.495  34940.510813   

       BB_lower  BB_middle        RSI      MACD  Signal_line  
4  34872.187578  34914.275  37.359551 -6.360879

In [28]:
# ヒストリカルボラティリティを計算して新たなカラムとして追加
window = 14  # ヒストリカルボラティリティのウィンドウ期間（例：14分）

df['Historical_volatility'] = calc.calculate_historical_volatility(df, 'Close', window)

# 結果を表示
print(df.tail())

      startTime     Open     High      Low    Close   Volume      turnover  \
4  1.699197e+12  34898.0  34899.5  34888.0  34891.0   16.133   562960.3632   
3  1.699197e+12  34891.0  34898.1  34874.9  34879.8  132.443  4620298.8349   
2  1.699197e+12  34879.8  34889.8  34879.8  34889.7    8.055    281000.552   
1  1.699197e+12  34889.7  34891.9  34886.5  34891.9   18.847   657552.5822   
0  1.699197e+12  34891.9  34891.9  34879.8  34882.3   46.626  1626579.7096   

            Timestamp  SMA_short_term  SMA_long_term      BB_upper  \
4 2023-11-05 15:11:00        34896.40      34914.275  34956.362422   
3 2023-11-05 15:12:00        34887.36      34910.875  34952.543662   
2 2023-11-05 15:13:00        34887.30      34908.195  34947.932744   
1 2023-11-05 15:14:00        34890.08      34905.660  34942.520956   
0 2023-11-05 15:15:00        34886.94      34903.495  34940.510813   

       BB_lower  BB_middle        RSI      MACD  Signal_line  \
4  34872.187578  34914.275  37.359551 -6.36087

In [29]:
# ストキャスティクスを計算して新たなカラムとして追加
k_window = 14  # %Kのウィンドウ期間（例：14分）
d_window = 3   # %Dのウィンドウ期間（例：3分）

df['%K'], df['%D'] = calc.calculate_stochastics(df, 'High', 'Low', 'Close', k_window, d_window)

# 結果を表示
print(df.tail())

      startTime     Open     High      Low    Close   Volume      turnover  \
4  1.699197e+12  34898.0  34899.5  34888.0  34891.0   16.133   562960.3632   
3  1.699197e+12  34891.0  34898.1  34874.9  34879.8  132.443  4620298.8349   
2  1.699197e+12  34879.8  34889.8  34879.8  34889.7    8.055    281000.552   
1  1.699197e+12  34889.7  34891.9  34886.5  34891.9   18.847   657552.5822   
0  1.699197e+12  34891.9  34891.9  34879.8  34882.3   46.626  1626579.7096   

            Timestamp  SMA_short_term  SMA_long_term      BB_upper  \
4 2023-11-05 15:11:00        34896.40      34914.275  34956.362422   
3 2023-11-05 15:12:00        34887.36      34910.875  34952.543662   
2 2023-11-05 15:13:00        34887.30      34908.195  34947.932744   
1 2023-11-05 15:14:00        34890.08      34905.660  34942.520956   
0 2023-11-05 15:15:00        34886.94      34903.495  34940.510813   

       BB_lower  BB_middle        RSI      MACD  Signal_line  \
4  34872.187578  34914.275  37.359551 -6.36087

In [30]:
# オンバランスボリュームを計算して新たなカラムとして追加
df['OBV'] = calc.calculate_obv(df, 'Close', 'Volume')

# 結果を表示
print(df.tail())

      startTime     Open     High      Low    Close   Volume      turnover  \
4  1.699197e+12  34898.0  34899.5  34888.0  34891.0   16.133   562960.3632   
3  1.699197e+12  34891.0  34898.1  34874.9  34879.8  132.443  4620298.8349   
2  1.699197e+12  34879.8  34889.8  34879.8  34889.7    8.055    281000.552   
1  1.699197e+12  34889.7  34891.9  34886.5  34891.9   18.847   657552.5822   
0  1.699197e+12  34891.9  34891.9  34879.8  34882.3   46.626  1626579.7096   

            Timestamp  SMA_short_term  SMA_long_term      BB_upper  \
4 2023-11-05 15:11:00        34896.40      34914.275  34956.362422   
3 2023-11-05 15:12:00        34887.36      34910.875  34952.543662   
2 2023-11-05 15:13:00        34887.30      34908.195  34947.932744   
1 2023-11-05 15:14:00        34890.08      34905.660  34942.520956   
0 2023-11-05 15:15:00        34886.94      34903.495  34940.510813   

       BB_lower  BB_middle        RSI      MACD  Signal_line  \
4  34872.187578  34914.275  37.359551 -6.36087

In [31]:
# 不要な列を削除
df = df.drop(['startTime', 'Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume', 'turnover'], axis=1).dropna()



array([[0.8810301 , 0.05181219, 0.06715771],
       [0.86817278, 0.05557926, 0.07624796],
       [0.87725184, 0.05217496, 0.07057319],
       [0.87280036, 0.05598036, 0.07121928],
       [0.8785032 , 0.04846005, 0.07303675],
       [0.83825586, 0.07579316, 0.08595098],
       [0.77489225, 0.12201052, 0.10309723],
       [0.79767604, 0.10868736, 0.09363661],
       [0.80228403, 0.09372376, 0.10399221],
       [0.78514137, 0.10179524, 0.11306339],
       [0.78460222, 0.10248236, 0.11291542],
       [0.80011354, 0.10229935, 0.09758711],
       [0.82949039, 0.09025061, 0.080259  ],
       [0.81980411, 0.09302968, 0.08716621],
       [0.84035975, 0.08938965, 0.0702506 ],
       [0.83836174, 0.09227802, 0.06936024],
       [0.81437562, 0.10698287, 0.07864151],
       [0.81789184, 0.10795121, 0.07415695],
       [0.83938511, 0.09240598, 0.06820891],
       [0.80074233, 0.09444995, 0.10480772],
       [0.84959966, 0.07035843, 0.08004191],
       [0.85795962, 0.07004665, 0.07199373],
       [0.

In [32]:
last_row = df.tail(1)
model.predict(last_row)

array([[0.82815395, 0.06934604, 0.10250001]])